# Introduction to `linopy`

:::{note}
This material is in part adapted from the following resources:
- [Linopy Getting Started](https://linopy.readthedocs.io/en/latest/index.html)
- [PyPSA simple electricity market examples](https://pypsa.readthedocs.io/en/latest/examples/simple-electricity-market-examples.html)
:::

<img src="https://github.com/PyPSA/linopy/blob/master/doc/logo.png?raw=true" width="300px" />

[Linopy](https://linopy.readthedocs.io/en/latest/index.html)  is an open-source framework for formulating, solving, and analyzing optimization problems with Python.

With Linopy, you can create optimization models within Python that consist of decision variables, constraints, and optimization objectives. You can then solve these instances using a variety of commercial and open-source solvers (specialised software).

[Linopy](https://linopy.readthedocs.io/en/latest/index.html) supports a wide range of problem types, including:

- **Linear programming**
- Integer programming
- Mixed-integer programming
- Quadratic programming


:::{note}
Documentation for this package is available at https://linopy.readthedocs.io.
:::

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install pandas linopy highspy
```
:::

## Solve a Basic Model

In this example, we explain the basic functions of the linopy Model class. First, we are setting up a very simple linear optimization model, given by

Minimize:
    $$x + 2y$$
subject to:
    $$ x \ge 0 $$
    $$y \ge 0 $$
    $$3x + 7y \ge 10 $$
    $$5x + 2y \ge 3 $$

### Initializing a `Model`

The Model class in Linopy is a fundamental part of the library. It serves as a container for all the relevant data associated with a linear optimization problem. This includes variables, constraints, and the objective function.

In [ ]:
import linopy

m = linopy.Model()

This creates a new Model object, which you can then use to define your optimization problem.

### Adding variables

Variables in a linear optimization problem represent the decision variables. A variable can always be assigned with a lower and an upper bound. In our case, both `x` and `y` have a lower bound of zero (default is unbouded). In Linopy, you can add variables to a Model using the `add_variables` method:

In [ ]:
x = m.add_variables(lower=0, name="x")
y = m.add_variables(lower=0, name="y");

`x` and `y` are linopy variables of the class `linopy.Variable`. Each of them contain all relevant information that define it. The `name` parameter is optional but can be useful for referencing the variables later.

In [ ]:
x

### Adding Constraints

Constraints define the feasible region of the optimization problem. They consist of the left hand side (lhs) and the right hand side (rhs). The first constraint that we want to write down is $3x + 7y = 10$ which we write out exactly in the mathematical way:

In [ ]:
3 * x + 7 * y >= 10

Note, we can also mix the constant and the variable expression, like this

In [ ]:
3 * x + 7 * y - 10 >= 0

… and linopy will automatically take over the separation of variables expression on the lhs, and constant values on the rhs.

The constraint is currently not assigned to the model. We assign it by calling the function `m.add_constraints`

In [ ]:
m.add_constraints(3 * x + 7 * y >= 10)
m.add_constraints(5 * x + 2 * y >= 3);

### Adding the Objective 

The objective function defines what you want to optimize. You can set the objective function of a Model in Linopy using the add_objective method. For our example that would be

In [ ]:
m.add_objective(x + 2 * y, sense="min")

Note, we can either minimize or maximize in Linopy. Per default, Linopy applies `sense='min'` making it not necessary to explicitly define the optimization sense.

### Solving the Model

Once you've defined your Model with variables, constraints, and an objective function, you can solve it using the `solve` method:

In [ ]:
m.solve()

The solution of the linear problem assigned to the variables under `solution` in form of a `xarray.Dataset`.

In [ ]:
x.solution

In [ ]:
y.solution

Well done! You solved your first linopy model!

## Use Coordinates

Now, the real power of the package comes into play! 

Linopy is structured around the concept that variables, and therefore expressions and constraints, have coordinates. That is, a `Variable` object actually contains multiple variables across dimensions, just as we know it from a `numpy` array or a `pandas.DataFrame`.

Suppose the two variables `x` and `y` are now functions of time `t` and we would modify the problem according to: 

Minimize:
$$\sum_t x_t + 2 y_t$$

subject to:

$$x_t \ge 0 \qquad \forall t $$
$$y_t \ge 0 \qquad \forall t $$
$$3x_t + 7y_t \ge 10 t \qquad \forall t$$
$$5x_t + 2y_t \ge 3 t \qquad \forall t$$

whereas `t` spans all the range from 0 to 10.

In order to formulate the new problem with linopy, we start again by initializing a model.

In [ ]:
m = linopy.Model()

Again, we define `x` and `y` using the `add_variables` function, but now we are adding a `coords` argument. This automatically creates optimization variables for all coordinates, in this case time-steps.

In [ ]:
import pandas as pd

time = pd.Index(range(10), name="time")

x = m.add_variables(
    lower=0,
    coords=[time],
    name="x",
)
y = m.add_variables(lower=0, coords=[time], name="y")

Following the previous example, we write the constraints out using the syntax from above, while multiplying the rhs with `t`. Note that the coordinates from the lhs and the rhs have to match. 

.. note::
    In the beginning, it is recommended to use explicit dimension names. Like that, things remain clear and no unexpected broadcasting (which we show later) will happen. 

In [ ]:
factor = pd.Series(time, index=time)

3 * x + 7 * y >= 10 * factor

It always helps to write out the constraints before adding them to the model. Since they look good, let's assign them.

In [ ]:
con1 = m.add_constraints(3 * x + 7 * y >= 10 * factor, name="con1")
con2 = m.add_constraints(5 * x + 2 * y >= 3 * factor, name="con2")
m

Now, when it comes to the objective, we use the `sum` function of `linopy.LinearExpression`. This stacks all terms all terms of the `time` dimension and writes them into one big expression. 

In [ ]:
obj = (x + 2 * y).sum()
m.add_objective(obj)

In [ ]:
m.solve()

In order to inspect the solution. You can go via the variables, i.e. `y.solution` or via the `solution` aggregator of the model, which combines the solution of all variables. This can sometimes be helpful.

In [ ]:
m.solution.to_dataframe().plot(grid=True, ylabel="Optimal Value");

Alright! Now you learned how to set up linopy variables and expressions with coordinates. For more advanced `linopy` operations you can check out the [User Guide](https://linopy.readthedocs.io/en/latest/user-guide.html).

## Electricity Market Examples

### Single bidding zone, single period

We want to minimise operational cost of an example electricity system representing South Africa subject to generator limits and meeting the load:

\begin{equation}
    \min_{g_s} \sum_s o_s g_s
  \end{equation}
  such that
  \begin{align}
    g_s &\leq G_s \\
    g_s &\geq 0 \\
    \sum_s g_s &= d
  \end{align}

We are given the following information on the South African electricity system:

Marginal costs in EUR/MWh

In [ ]:
marginals_df = pd.DataFrame(
    {"Generator": ["Wind", "Coal", "Gas", "Oil"], "MarginalCost": [0, 30, 60, 80]}
)

Power plant capacities in MW

In [ ]:
capacities_df = pd.DataFrame(
    {"Generator": ["Wind", "Coal", "Gas", "Oil"], "Capacity": [3000, 35000, 8000, 2000]}
)

Inelastic demand in MW

In [ ]:
load = 42000

We now start building the model

In [ ]:
m = linopy.Model()

Let's define the dispatch variables `g` with the `lower` and `upper` bound:
$$g_s \leq G_s $$
$$g_s \geq 0 $$

In [ ]:
g = m.add_variables(
    lower=0, upper=capacities_df.Capacity, coords=[capacities_df.Generator], name="g"
)
g

And and the objective:
$$\min_{g_s} \sum_s o_s g_s$$

In [ ]:
m.add_objective(marginals_df.MarginalCost.values * g, sense="min")
m.objective

Which is subject to: 

$$\sum_s g_s = d$$

In [ ]:
m.add_constraints(g.sum() == load, name="energy_balance")

It always helps to write out the constraints before adding them to the model. Since they look good, let’s assign them.

In [ ]:
m.solve()

This is the optimimal generator dispatch (MW)

In [ ]:
m.solution.to_dataframe()

And the market clearing price we can read from the shadow price of the energy balance constraint (i.e. the added cost of increasing electricity demand by one unit):

In [ ]:
m.dual["energy_balance"].item()

### Two bidding zones with transmission

Let's add a spatial dimension, such that the optimisation problem is expanded to
\begin{equation}
  \min_{g_{i,s}, f_\ell} \sum_s o_{i,s} g_{i,s}
\end{equation}
such that
\begin{align}
  g_{i,s} &\leq G_{i,s} \\
  g_{i,s} &\geq 0 \\
  \sum_s g_{i,s} - \sum_\ell K_{i\ell} f_\ell &= d_i & \text{KCL} \\
  |f_\ell| &\leq F_\ell & \text{line limits}  \\
  \sum_\ell C_{\ell c} x_\ell f_\ell &= 0 & \text{KVL} 
\end{align}

In this example, we connect the previous South African electricity system with a hydro generation unit in Mozambique through a single transmission line. Note that because a single transmission line will not result in any cycles, we can neglect KVL in this case.

We are given the following data (all in MW):

In [ ]:
generators = ["Coal", "Wind", "Gas", "Oil", "Hydro"]
countries = ["South Africa", "Mozambique"]

capacities_df = pd.DataFrame(
    {
        "Coal": [35000, 0],
        "Wind": [3000, 0],
        "Gas": [8000, 0],
        "Oil": [2000, 0],
        "Hydro": [0, 1200],
    },
    index=countries,
)

marginal_df = pd.DataFrame({"MarginalCost": [30, 0, 60, 80, 0]}, index=generators)

loads_df = pd.DataFrame({"Load": [42000, 650]}, index=countries)

transmission = 500

Let's start with a new model instance

In [ ]:
m = linopy.Model()

Now we create dispatch variables, as before, with the `upper` and `lower` bound for each countries and generators only when the `upper` is greater than 0.

In [ ]:
for country in countries:
    for generator in generators:
        upper_bound = capacities_df.loc[country, generator]
        if upper_bound > 0:
            var_name = f"g_{generator}_{country}"
            m.add_variables(lower=0, upper=upper_bound, name=var_name)

We als need an additional variable for the flow.

In [ ]:
m.add_variables(name="line_limit")

The objective can be written as:
$$\min_{g_{i,s}, f_\ell} \sum_s o_{i,s} g_{i,s}$$